### Notation

| symbol | set | code | description |
|:------:|:---:|:----:|:------------|
| $f$    | $\mathbb{R}$ | `myfunc` | the function |
| $\mathbf{X}$ | $\mathbb{R}^n$ | `X` | vector with variables |
| $x_i$ | $\mathbb{R}$ | `x[i]` | the $i$-th variable  |
| $h$ | $\{h \in \mathbb{R} \| h>0 \}$ | `h` | increment parameter (step size, band width) |
| $\frac{\partial f(\mathbf{X})}{\partial x_i}$ | $\mathbb{R}$ | `g[i]` | partial derivative for the $i$-th variable |
| $\nabla f$ | $\mathbb{R}^n$ | `g` | gradient vector |


### Central Difference Approximation
The **central difference approximation** (symmetric difference quotient) addresses these issue, 
that forward and backward difference approximation are never compute in point $\mathbf{X}$.
The partial derivative approximation or gradient $g^c_i$ lies around point $\mathbf{X}$, 
and a smaller bias toward bigger or smaller gradients depending on the curvature.

$$
\frac{\partial f(\mathbf{X})}{\partial x_i} 
\approx g^c_i 
= \frac{f(\mathbf{X} + h \cdot 1_i) - f(\mathbf{X} - h \cdot 1_i)}{2 \cdot h}
$$



### C Implementation
The central difference approximation requires `n*2` function evaluations and thus is higher than for forward or backward approximation `n+1`.

The `x` array is temporarily manipulated in `finitediff_central` two times.
First, the `x[i]` value is backed up as `tmp`.
Second, the bandwidth `h` is **added** to compute `f0`.
Aftwards, the backup `tmp` **minus** `h` is assigned to `x[i]` to compute `f1`.
Fourth, `tmp` is reassigned to `x[i]`.
And finally, `g[i]` is computed with `f0` and `f1`.

In [1]:
#include <stdio.h>  

/**************************************************
 ************* finitediff_central.h ***************
 **************************************************/
void finitediff_central(double (*f)(double*, int), 
                        double *x, 
                        int n, 
                        double h, 
                        double *g){
    double tmp, f0, f1;
    double h2 = 2.0*h;
    for (int i=0; i<n; i++){
        tmp = x[i]; //store x[i] temporarly
        x[i] += h;  //add bandwidth param
        f0 = f(x,n); //left 
        x[i] = tmp - h; //reassign x[i] minus bandwith
        f1 = f(x,n); //right
        x[i] = tmp;  //reassign orginal x[i] for next i
        g[i] = (f0 - f1) / h2; //difference
    }
} 


/**************************************************
 **************** EXAMPLE SCRIPT ******************
 **************************************************/
double myfunc(double *x, int n){
    double f = 0.0;
    for(int i=0; i<n; i++){
        f += x[i] * x[i];
    }
    f /= (double)n;
    return f;
}

int main(){
    double (*evalfunc)(double*, int) = &myfunc;
    double x[]={-.6, -.5, -.4, -.3, -.2, -.1, 
                0.0, .1 , .2, .3, .4, .5, .6};
    int n = 13;
    double g[n];
    double h = 0.001;
    
    finitediff_central(evalfunc, x, n, h, g);
    
    for(int i=0; i<n; i++){
        printf("x: %7.4f | g(x): %7.4f \n", x[i], g[i] );
    }
    
    return 0;
}


x: -0.6000 | g(x): -0.0923 
x: -0.5000 | g(x): -0.0769 
x: -0.4000 | g(x): -0.0615 
x: -0.3000 | g(x): -0.0462 
x: -0.2000 | g(x): -0.0308 
x: -0.1000 | g(x): -0.0154 
x:  0.0000 | g(x):  0.0000 
x:  0.1000 | g(x):  0.0154 
x:  0.2000 | g(x):  0.0308 
x:  0.3000 | g(x):  0.0462 
x:  0.4000 | g(x):  0.0615 
x:  0.5000 | g(x):  0.0769 
x:  0.6000 | g(x):  0.0923 
